In [1]:
import pandas as pd
import numpy as np
import nltk
import os

In [2]:
from lfs.helper import *
from lfs.lf_categorical import *
from lfs.lf_extration import *
from lfs.lf_numerical import *
from lfs.lf_unusable import *

In [3]:
import os
number_of_files = 1000
base_dir = 'data/labels/'
ext = '.xlsx'
df = pd.DataFrame()
for file_name in os.listdir(base_dir):
    if number_of_files < 0: break
    if not file_name.endswith(ext): continue
    number_of_files -= 1
    print('reading files', file_name)
    df = df.append(pd.read_excel(base_dir + file_name))

reading files data_for_labelling4.xlsx
reading files data_for_labelling8.xlsx
reading files data_for_labelling9.xlsx
reading files data_for_labelling5.xlsx
reading files data_for_labelling3.xlsx
reading files data_for_labelling2.csv.xlsx
reading files data_for_labelling6 (1) (1).xlsx
reading files data_for_labelling1.csv.xlsx
reading files data_for_labelling0.csv.xlsx
reading files data_for_labelling7.xlsx


In [4]:
cols = df.columns.tolist()
print(cols)

['% nans', 'Attribute_name', 'Check', 'Num of nans', 'Reason', 'Record_id', 'Total_val', 'Unnamed: 2', 'Unnamed: 9', 'check', 'max_val', 'mean', 'min_val', 'num of dist_val', 'reason', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'std_dev', 'y_Arun', 'y_act', 'y_pred']


In [5]:
cols = ['y_act','Reason','Attribute_name','Total_val','num of dist_val','% nans'] + cols[10:13] + ['std_dev'] + cols[15:-4]

In [6]:
df = df.reindex(columns=cols)
df[:1]

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
0,Unusable,l,StratificationCategory2,403984,1,80.364816,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [7]:
print(df.columns.tolist())

['y_act', 'Reason', 'Attribute_name', 'Total_val', 'num of dist_val', '% nans', 'max_val', 'mean', 'min_val', 'std_dev', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']


In [8]:
df.y_act.unique()

array(['Unusable', 'Usable with extraction',
       'Usable directly categorical', 'Context_specific',
       'Usable directly numeric', 'Usable Directly Numeric',
       'Usable Directly Categorical', 'Usable with Extraction',
       'Usable Directly Categorical ', 'Usable with Extration',
       'Usable With Extraction', 'Usable with Extraction ', 'unusable',
       'Usable Directly numeric', 'Usable directly Categorical',
       'Usable Directly categorical'], dtype=object)

In [9]:
df['y_act'] = df.apply(lambda row: label_mapper(row), axis=1)

In [10]:
categories = {'Usable Directly Categorical':1, 'Usable Directly Numeric':2, 'Usable With Extraction':3, 'Unusable':4, 'Context_Specific':5, 'None':0}

In [11]:
categories

{'Context_Specific': 5,
 'None': 0,
 'Unusable': 4,
 'Usable Directly Categorical': 1,
 'Usable Directly Numeric': 2,
 'Usable With Extraction': 3}

In [12]:
categories_size = {'Context_Specific':2050,'Unusable':891,'Usable Directly Categorical':2087,'Usable Directly Numeric':5063,'Usable With Extraction':650}

In [13]:
df.groupby(df.y_act).size()

y_act
Context_Specific               2050
Unusable                        891
Usable Directly Categorical    2087
Usable Directly Numeric        5063
Usable With Extraction          650
dtype: int64

In [14]:
catelens = [0, 2087, 5063, 650, 891, 2050]

In [15]:
find_key_word('job', df, 'Attribute_name')

[0, 44, 4, 13, 17, 0]

In [16]:
find_key_word('label', df, 'Attribute_name')

[0, 14, 1, 1, 5, 0]

In [17]:
find_key_word('id', df, 'Attribute_name')

[0, 58, 45, 11, 115, 122]

In [18]:
find_key_word('block', df, 'Attribute_name')

[0, 6, 0, 0, 0, 0]

# Usable With Extraction

|Total|Matched|Mismatched| Abstained|
| --- | --- | --- | --- |
|   650  |  581 | 559  | 69|

In [20]:
lfs_ext = [lf_extractable_units, lf_extractable_number_sci, lf_extractable_pattern, lf_date_extraction_name, lf_extractable_name, lf_extractable_list, lf_extractable_sample_length, lf_date_extraction_samples, lf_extractable_email_url]
len(lfs_ext)

9

In [21]:
cms = test_lfs(lfs_ext, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_extractable_units          	         0         1         0         6         0         2
lf_extractable_number_sci     	         0         1         1         0         0         0
lf_extractable_pattern        	         0        25         1        78        14        14
lf_date_extraction_name       	         0         6        21       149        12         1
lf_extractable_name           	         0        84        61       182        52        11
lf_extractable_list           	         0         2         3        27         5         0
lf_extractable_sample_length  	         0       123         3       274        37        57
lf_date_extraction_samples    	         0        19         0       264        17        51
lf_extractable_email_url      	         0         0         0        33        10         4


In [22]:
cm, abstained, mismatched = test_lfs_onecategory(lfs_ext, df, 3)
cm, len(abstained), len(mismatched)

([0, 225, 86, 581, 123, 125], 69, 559)

In [23]:
catelens

[2087, 5063, 650, 891, 2050, 0]

In [24]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/extractable_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
485,Unusable,f,category,9234,1,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,gross_domestic_product_gdp,gross_domestic_product_gdp,gross_domestic_product_gdp,gross_domestic_product_gdp,gross_domestic_product_gdp
328,Usable Directly Numeric,NaN,install_size_kw_buckets,48722,48530,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,[[0171][5231][10114][1545][2016][252][352][451...,[[060][550][1024][1518][2023][2515][3012][3515...,[[0364][5313][1057][1513][207][258][305][351][...,[[0320][567][1020][1512][251][302][351][401][4...,[[0125][5219][10227][15124][2039][2520][3015][...
79,Usable Directly Categorical,i,consume,258,42,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5,"4,2","5,5","3,9","4,5"
360,Usable Directly Numeric,NaN,OTHER_EXPENDITURE,1224,1172,NaN,3.759373e+06,4.231043e+05,1.154100e+04,5.246462e+05,NaN,237222,36291,164083,68617
25,Context_Specific,m,r.sourceID,364456,5,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Paradise Papers - Appleby,Paradise Papers - Malta corporate registry,Paradise Papers - Barbados corporate registry,Paradise Papers - Bahamas corporate registry,Paradise Papers - Lebanon corporate registry
62,Usable Directly Categorical,h,Other groups,263,2,77.56654,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,HIPC,Euro area,HIPC,HIPC
336,Usable Directly Numeric,NaN,Other Investing Activities,1781,1471,NaN,1.160200e+10,-4.054141e+08,-5.671700e+10,3.186742e+09,11000000,481000000,601000000,114000000,-1796000
82,Context_Specific,n,contentId,72312,2987,0.00000,9.220000e+18,-3.030000e+16,-9.220000e+18,5.340000e+18,-3500000000000000000,8890000000000000000,311000000000000000,-7820000000000000000,-8860000000000000000
333,Usable Directly Numeric,NaN,Other Current Liabilities,1781,1046,NaN,1.360000e+12,1.500814e+10,0.000000e+00,1.090000e+11,4524000000,7385000000,7059000000,6272000000,149558000
508,Usable Directly Categorical,f,category,595971,15,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,agriculture_hunting_and_related_service_activi...,agriculture_hunting_forestry_fishing_a_b,construction_f,education_health_and_social_work_other_communi...,electricity_gas_and_water_supply_e


In [25]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/extractable_abstained.csv')
abstained.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
32,Usable With Extraction,NaN,ICO,768922,430118,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,% TONY GARCIA,% STEVEN FELDMAN,% JAYNE PALMER,% THOMAS C PUCKETT
9,Usable With Extraction,c,Location,806179,490,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"Manhattan, NY","Boston, MA",None,"Philadelphia, PA","Toronto, CANADA"
53,Usable With Extraction,c,LocationName,2772,984,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"Badakhshan, Afghanistan","Badghis, Afghanistan","Baghlan, Afghanistan","Balkh, Afghanistan","Bamyan, Afghanistan"
22,Usable With Extraction,NaN,Language;Country,93,93,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,fi;FIN,en;USA,de;GER,da;DNK,he;ISR
34,Usable With Extraction,NaN,ICO,3324,2050,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,% RODOLFO O FONT,% TATSUHIKO WAKAO,% CARMEN M CRUZ RIVERA,% ROY MORRIS ESQ
41,Usable With Extraction,c,mpi_region,4861,26,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,"Barranquilla A. M., Colombia","Medellin A.M., Colombia","Santanderes, Colombia","Cali A.M., Colombia","Nord, Haiti"
61,Usable With Extraction,i,value,225363,160847,0.369626,1.450000e+18,3.789170e+13,-4.706850e+13,4.781780e+15,275931000000,266121000000,250767000000,235011000000,224056000000
26,Usable With Extraction,NaN,ICO,1973,1269,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,% TATSUHIKO WAKAO,% ROY MORRIS ESQ,% KAREN STEINFELD SECRETARY,% CARR RIGGS & INGRAM LLC
19,Usable With Extraction,c,topic_name,2509,2509,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Sports Fan,Latin Music,English as a Second Language,Spanish Language,Italian Language
20,Usable With Extraction,NaN,dt_desc,11106,197,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Balance,Exports,Imports,Rental Vacancy Rate,Homeowner Vacancy Rate


# Usable Directly Categorical

|Total|Matched|Mismatched| Abstained| Accuracy | Coverage |
| --- | --- | --- | --- | --- | --- |
|   2087  |  1482 | 823  | 605 | .6430    | .7101    |
|   2087  |  1788 | 1514  | 299 | .5415    | .8567    |

In [15]:
lfs_cate = [lf_binary_category, lf_name_category, lf_coded_abbreviation, lf_finite_set_name, lf_finite_set_sample, lf_dist_string_percentage, lf_dist_num_percentage]
len(lfs_cate)

7

In [16]:
cms = test_lfs(lfs_cate, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_binary_category            	         0       928        46         7       101       154
lf_name_category              	         0       330       100        22        48        11
lf_coded_abbreviation         	         0       103         0         0        27        11
lf_finite_set_name            	         0       381        13        63        99        27
lf_finite_set_sample          	         0       226         1        83        64        79
lf_dist_string_percentage     	         0       808       161       210       100       185
lf_dist_num_percentage        	         0       515       143         0        17       265


In [17]:
cm, abstained, mismatched = test_lfs_onecategory(lfs_cate, df, 1)
cm, len(abstained), len(mismatched)

([0, 1788, 419, 291, 275, 529], 299, 1514)

In [18]:
catelens

[0, 2087, 5063, 650, 891, 2050]

In [19]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/cate_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
393,Usable With Extraction,NaN,owner,109979.0,1032.0,NaN,0.0,0.000000,0.0,0.000000,amit@chromium.org,jon@chromium.org,pfeldman@chromium.org,pkasting@chromium.org,brettw@chromium.org
132,Usable With Extraction,d,recr_info_prof_org_advert_to,374362.0,990.0,92.497636,0.0,0.000000,0.0,0.000000,NaN,27/06/14,2015-04-02 00:00:00,31/12/14,26/12/14
522,Usable With Extraction,c,q0027_other,25090.0,670.0,NaN,0.0,0.000000,0.0,0.000000,statisics - using R,NaN,Algorithms and data analysis,I didn't know about it,"Bid data, Neural networks"
489,Context_Specific,i,s1p2wat3,9597.0,4.0,65.582995,4.0,0.030881,0.0,0.305787,NaN,4,0,1,3
753,Usable With Extraction,NaN,Address of Headquarters,505.0,264.0,NaN,0.0,0.000000,0.0,0.000000,"St. Paul, Minnesota","North Chicago, Illinois","Dublin, Ireland","Santa Monica, California","Atlanta, Georgia"
521,Context_Specific,i,s3p1wat2,9597.0,2.0,73.043659,2.0,0.021647,0.0,0.206942,NaN,0,2,2,0
150,Usable With Extraction,d,ri_employee_referral_prog_to,374362.0,1783.0,81.642902,0.0,0.000000,0.0,0.000000,NaN,2013-01-10 00:00:00,24/07/13,27/02/14,19/07/13
1032,Usable Directly Numeric,a,Y1_ActualVelocity,1301.0,189.0,NaN,12.7,-0.607375,-32.7,6.539193,0,-0.025,0.025,12.7,-0.05
129,Usable With Extraction,d,recr_info_pro_org_advert_from,374362.0,713.0,96.240003,0.0,0.000000,0.0,0.000000,NaN,2013-01-10 00:00:00,2013-01-09 00:00:00,30/09/13,2013-12-08 00:00:00
762,Usable Directly Numeric,a,UpperCorridor,2720.0,11.0,89.117647,1.5,1.199764,0.8,0.100976,NaN,1.2,1.25,1.1,1.12


In [20]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/cate_abstained.csv')
abstained.sample(100)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
639,Usable Directly Categorical,g,Block,89830.0,3032.0,NaN,9999.0,4.130837e+03,0.00,1.018353e+03,2260,2261,0,2302,2362
124,Usable Directly Categorical,i,YearBuilt,89830.0,145.0,NaN,2016.0,1.805109e+03,0.00,4.999684e+02,0,1931,1920,1967,1963
940,Usable Directly Categorical,NaN,ZIP,3324.0,1745.0,NaN,0.0,0.000000e+00,0.00,0.000000e+00,00000-0000,00984-3869,00909-0000,96799-1715,00927-5131
427,Usable Directly Categorical,a,HrIi,1518.0,4.0,0.000000,3.0,5.928854e-02,0.00,3.725283e-01,0,3,1,2,1
377,Usable Directly Categorical,a,Terps,1518.0,4.0,0.000000,3.0,2.951252e-01,0.00,8.123649e-01,0,3,1,2,2
410,Usable Directly Categorical,g,AdministeringGovt,2720.0,4.0,0.000000,5.0,7.176471e-01,0.00,1.213762e+00,0,5,2,1,0
107,Usable Directly Categorical,i,against_rock,801.0,5.0,NaN,4.0,1.250312e+00,0.25,6.967128e-01,1,2,4,0.5,0.25
1016,Usable Directly Categorical,a,adm2,9597.0,405.0,0.000000,0.0,0.000000e+00,0.00,0.000000e+00,Tikare,Koper,Tiankoura,Zabre,Sabou
176,Usable Directly Categorical,NaN,Permit Number,198900.0,181495.0,NaN,0.0,0.000000e+00,0.00,0.000000e+00,202000000000,202000000000,202000000000,202000000000,202000000000
900,Usable Directly Categorical,g,Police District,64618.0,102.0,NaN,0.0,0.000000e+00,0.00,0.000000e+00,590,822,830,850,871


In [73]:
from collections import Counter
words=[1,2,2,2,1]
Counter(words)

Counter({1: 2, 2: 3})

In [78]:
def lf_num_of_same_length(row):
    samples = get_samples(row)
    samples_digits = [sample.isdigit() for sample in samples]
    if sum(samples_digits) < 3:
        return 0
    samples_length = [len(sample) for sample in samples]
    for length in samples_length:
        flag = 0
        if length < 3: continue
        for _length in samples_length:
            if length == _length:
                flag += 1
        if flag > 3:
            return 1
    return 0

In [79]:
test_lf(lf_num_of_same_length, df)

[0, 154, 880, 20, 55, 143]

# Usable Directly Numeric

|Total|Matched|Mismatched| Abstained|
| --- | --- | --- | --- |
|   5063  |  5055 | 3459  | 8|

In [32]:
lfs_num = [lf_cast_to_numbers]

In [33]:
cms = test_lfs(lfs_num, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_cast_to_numbers            	         0      1092      5055        24       596      1747


In [34]:
cm, abstained, mismatched = test_lfs_onecategory(lfs_num, df, 2)
cm, len(abstained), len(mismatched)

([0, 1092, 5055, 24, 596, 1747], 8, 3459)

In [35]:
catelens

[2087, 5063, 650, 891, 2050, 0]

In [36]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/num_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
129,Context_Specific,o,1970,401016,61398,80.051170,8.060000e+14,2.100000e+11,-3.200000e+14,8.530000e+12,NaN,54.8221,43.3511,65.8262,0.771415
789,Context_Specific,a,s1p2c2,9597,46,88.131708,5.600000e+01,2.774802e+01,2.000000e+00,1.608523e+01,41,44,NaN,14,56
2764,Context_Specific,n,Y2003,21477,1935,NaN,3.548500e+05,4.726933e+02,0.000000e+00,4.911100e+03,3095,460,48,144,63
486,Context_Specific,a,othrhrm2,9597,17,43.190580,5.000000e+02,2.472487e-01,0.000000e+00,6.928319e+00,NaN,0,2,1,22
3289,Context_Specific,a,educ32,9597,2,99.947900,9.000000e+00,1.800000e+00,0.000000e+00,3.600000e+00,NaN,False,9,9,0
2949,Usable Directly Categorical,i,base_year,223379,60,0.354107,2.014000e+03,1.868014e+03,-1.000000e+00,4.860071e+02,2003,2006,2004,1993,-1
386,Context_Specific,a,landhhfd3,9597,24,72.439304,2.750000e+02,6.056711e-01,0.000000e+00,6.101712e+00,NaN,0,5,6,12
103,Unusable,l,n.ibcRUC,2031,0,100.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
1304,Usable Directly Categorical,i,ad746,9597,3,32.239241,1.000000e+00,1.324004e-01,-9.900000e+01,1.279549e+00,0,1,NaN,-99,1
2192,Context_Specific,NaN,Wfgm,914,30,NaN,4.400000e+01,2.604486e+01,1.300000e+01,4.793226e+00,32,31,29,27,17


In [37]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/num_abstained.csv')
abstained

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
0,Usable Directly Numeric,a,LOCATION,310579,285,5.988492,0.0,0.000000,0.0,0.000000,NaN,"(41.97562073966174, -87.72838313514907)","(41.90999558160536, -87.72639469321471)","(41.95302475075328, -87.78668323766634)","(41.83773267497158, -87.66589444999927)"
1,Usable Directly Numeric,a,distance,258,126,0.000000,0.0,0.000000,0.0,0.000000,28,12,"11,2","12,9","18,5"
2,Usable Directly Numeric,a,temp_inside,258,11,3.488372,0.0,0.000000,0.0,0.000000,"21,5","22,5",20,NaN,21
3,Usable Directly Numeric,a,average_review_rating,10000,19,NaN,0.0,0.000000,0.0,0.000000,4.9 out of 5 stars,4.5 out of 5 stars,3.9 out of 5 stars,5.0 out of 5 stars,4.7 out of 5 stars
4,Usable Directly Numeric,NaN,Main Source of Income,41544,3,NaN,0.0,0.000000,0.0,0.000000,Wage/Salaries,Other sources of Income,Enterpreneurial Activities,Wage/Salaries,Wage/Salaries
5,Usable Directly Numeric,NaN,install_size_kw_buckets,48722,48530,NaN,0.0,0.000000,0.0,0.000000,[[0171][5231][10114][1545][2016][252][352][451...,[[060][550][1024][1518][2023][2515][3012][3515...,[[0364][5313][1057][1513][207][258][305][351][...,[[0320][567][1020][1512][251][302][351][401][4...,[[0125][5219][10227][15124][2039][2520][3015][...
6,Usable Directly Numeric,NaN,location,188462,160187,NaN,0.0,0.000000,0.0,0.000000,(37.7216893346731 -122.430482526944),(37.8042860176436 -122.435150585742),(37.7647299507493 -122.446671573844),(37.7697365039652 -122.417760336994),(37.7583327223052 -122.412354747671)
7,Usable Directly Numeric,a,educ37,9597,2,99.968740,2.0,0.666667,0.0,0.942809,NaN,False,2,False,NaN


# Unusable

|Total|Matched|Mismatched| Abstained|
| --- | --- | --- | --- |
|   891  |  856 | 576  | 35|

In [38]:
lfs_unusable = [lf_unusable, lf_unusable_nans]

In [39]:
cms = test_lfs(lfs_unusable, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_unusable                   	         0        51       191        80       786       219
lf_unusable_nans              	         0        30         2         0       337        12


In [40]:
cm, abstained, mismatched = test_lfs_onecategory(lfs_unusable, df, 4)
cm, len(abstained), len(mismatched)

([0, 79, 192, 80, 856, 225], 35, 576)

In [41]:
catelens

[2087, 5063, 650, 891, 2050, 0]

In [42]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/unusable_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
251,Usable Directly Categorical,e,q26FrameLearnExpressJS,25090,2,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Will Learn,NaN,Know,NaN,NaN
515,Context_Specific,a,farmwork23,9597,2,99.301865,2.000000e+00,1.507463e+00,1.000000,4.999443e-01,NaN,2,1,NaN,2
247,Usable Directly Categorical,i,q23FrameBackboneDotJS,25090,3,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,0,1,NaN,NaN
208,Usable Directly Numeric,NaN,yearly_sunlight_kwh_e,52,52,NaN,2.585561e+10,2.580733e+09,0.000000,4.228606e+09,0,2420185385,4.49072e+07,6054636897,1020895160
0,Usable Directly Categorical,e,ji_live_in_dom_svc_contract,374362,3,99.359444,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,A,Y,N,N
238,Usable Directly Numeric,NaN,SUPPORT_SERVICES_EXPENDITURE,1224,1224,NaN,2.350185e+07,2.631397e+06,139963.000000,3.280875e+06,735036,350902,1007732,483488,8520926
194,Usable Directly Numeric,NaN,lng_avg,52,52,NaN,-6.988566e+01,-9.364726e+01,-157.507901,1.899155e+01,-116.635,-86.8507,-148.508,-111.895,-92.6524
24,Context_Specific,a,s1p2c5,9597,24,99.322705,5.600000e+01,3.413846e+01,4.000000,1.729907e+01,NaN,33,29,14,12
325,Context_Specific,n,z,63984,63959,NaN,2.184194e+01,1.574555e-03,-19.199339,8.976619e-01,0.0671728,0.108988,-0.0105302,-0.0895059,-0.284885
206,Usable Directly Numeric,NaN,yearly_sunlight_kwh_n,52,52,NaN,1.215200e+10,1.055046e+09,4745.559338,2.273631e+09,4745.56,1128360422,2.18659e+06,1780582587,455410026


In [43]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/unusable_abstained.csv')
abstained.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
32,Unusable,l,q23FramePadrino,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Padrino,Padrino,Padrino
16,Unusable,l,ZoneDist4,324403,6,NaN,0.0,0.000000,0.0,0.000000,NaN,R5,M3-1,R7B,R2
7,Unusable,k,ad7521,9597,7,23.997082,0.0,0.000000,0.0,0.000000,0,1,0,1,intercropping
10,Unusable,k,isbn,10000,9300,7.000000,0.0,0.000000,0.0,0.000000,439023483,439554934,316015849,61120081,743273567
27,Unusable,l,q22LangProfPython,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Python,Python,Python
30,Unusable,l,q23FrameStruts,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Struts,Struts,Struts
14,Unusable,l,ZoneDist4,89830,2,NaN,0.0,0.000000,0.0,0.000000,NaN,R5,R6A,R5,R6A
25,Unusable,l,q21CoreCompDatabaseDesign,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Database Design,Database Design,Database Design
24,Unusable,l,q17HirChaNoDiversCandidates,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,Not enough diverse candidates,NaN,Not enough diverse candidates,Not enough diverse candidates
3,Unusable,k,Sentence_Index,662207,27,0.000000,26.0,3.697087,0.0,3.129019,0,1,2,3,4


# Context_Specific

In [44]:
df[(df.y_act=='Context_Specific')].sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
211,Context_Specific,a,lvs3graze1,9597,12,42.752944,12.0,0.396250,0.0,2.016422,0,NaN,12,3,9
165,Context_Specific,NaN,Team,2150,297,NaN,1463.0,1291.615814,1102.0,103.268764,1207,1210,1228,1260,1374
794,Context_Specific,a,s2p1c3qharv,9597,204,63.665729,8000000.0,7353.813651,0.0,172758.422900,NaN,100,90,105,200
916,Context_Specific,a,s2p2c6sold,9597,5,73.887673,300.0,0.323224,0.0,8.376567,NaN,0,200,300,50
631,Context_Specific,o,CODIGO_COR_RACA,21124,5,0.000000,5.0,1.813482,1.0,0.956523,3,1,2,4,5
1042,Context_Specific,NaN,ENROLL,359836,23399,NaN,1077381.0,3123.834339,0.0,14008.465000,7568,19961,2045,2972,3539
1063,Context_Specific,a,hm15owner,9597,4,68.219235,3.0,0.236393,0.0,0.486729,NaN,3,1,2,0
13,Context_Specific,NaN,ADM1DHS,4861,28,NaN,9999.0,1945.311664,1.0,3948.520276,8,13,70,23,68
728,Context_Specific,a,s1p2c4area,9597,41,98.582890,70.0,10.446544,0.0,9.809756,5,15,NaN,14,12
1061,Context_Specific,a,hm15,9597,14,58.518287,1000.0,0.576488,0.0,16.074431,0,NaN,10,1,3
